In [1]:
import numpy as np  # library to handle data in a vectorized manner
import pandas as pd  # library for data analsysis
from bs4 import BeautifulSoup
import json  # library to handle JSON files
import requests  # library to handle requests
from urllib import request
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

print("Libraries imported.")


Libraries imported.


In [ ]:
# The CSV files is exported from my Web Scraping of School informations.ipynb file
school = pd.read_csv("school_scores.csv")
school = school.iloc[:, 1:]

# Since the origin csv file includes all of the schools in Ontario, here we need to define the Cities belongs to GTA
# Due to project objective, I didn't selected all of the cities in GTA
GTA = [
    "Toronto",
    "Mississauga",
    "Ajax",
    "Brampton",
    "Unionville",
    "Oakville",
    "Richmond Hill",
    "Markham",
    "Oshawa",
    "Whitby",
    "Caledon",
    "Pickering",
    "Maple",
    "Thornhill",
    "Woodbridge",
    "Aurora",
    "Vaughan",
    "Concord",
    "Stouffville",
    "Milton",
    "Newmarket",
    "King City",
    "Durham",
]
school = school[school["school_city"].isin(GTA)]
school.head()


In [ ]:
# Scraping House informations close to the school
n = 0
address_list = []
num_bedroom_list = []
num_bathroom_list = []
price_list = []
post_code_list = []
latitude_list = []
longitude_list = []
neighborhood_list = []
ptype_list = []
size_list = []

for post_code in school.school_post_code:
    n += 1
    for ptype in ["house", "condo", "townhouse"]:
        url = "for Security issue, here I just delete the url link of the listing website".format(
            post_code, ptype
        )
        print("loading school number:", n, "for house type", ptype, url)
        # Fetch the html file
        html_doc = requests.get(url).text
        soup = BeautifulSoup(html_doc)

        headings = soup.findAll(
            "ul",
            {
                "class": "headings-menu list-unstyled xs-flex xs-flex-wrap xs-text-5 sm-mt1"
            },
        )[0]
        containers1 = soup.findAll("li", {"class": "listing-column text-4"})
        containers2 = soup.findAll("span", {"itemprop": "geo"})
        containers3 = soup.findAll("span", {"class": "neighbourhood"})
        containers4 = soup.findAll("span", {"class": "xs-mr05"})

        if (
            headings.findAll("li", {"class": "xs-pr1 xs-mt1 xs-hide sm-block"})[0]
            .contents[0]
            .contents[0]
            == "Zero Homes for Sale"
        ):
            print("Zero Homes for Sale")
            continue

        else:
            for i in range(len(containers1)):
                try:
                    address = (
                        containers1[i]
                        .findAll("span", {"class": "street"})[0]
                        .contents[0]
                    )
                    num_bedroom = (
                        containers1[i]
                        .findAll("li", {"class": "xs-inline xs-mr05"})[0]
                        .contents[0]
                    )
                    num_bathroom = (
                        containers1[i]
                        .findAll("li", {"class": "xs-inline xs-mr05"})[1]
                        .contents[1]
                    )

                    try:
                        size = (
                            containers1[i]
                            .findAll("li", {"class": "xs-inline xs-mr05"})[2]
                            .contents[1]
                        )
                    except:
                        size = ""

                    price = (
                        containers1[i]
                        .findAll("span", {"itemprop": "price"})[0]
                        .contents[0]
                    )

                    latitude = float(
                        str(containers2[i].findAll("meta")[0])
                        .strip()
                        .split()[1]
                        .strip("content=")
                        .strip('"')
                    )
                    longitude = float(
                        str(containers2[i].findAll("meta")[1])
                        .strip()
                        .split()[1]
                        .strip("content=")
                        .strip('"')
                    )

                    neighborhood = containers3[2].contents[1]

                    address_list.append(address)
                    num_bedroom_list.append(num_bedroom)
                    num_bathroom_list.append(num_bathroom)
                    price_list.append(price)
                    post_code_list.append(post_code)
                    latitude_list.append(latitude)
                    longitude_list.append(longitude)
                    neighborhood_list.append(neighborhood)
                    ptype_list.append(ptype)
                    size_list.append(size)

                    print(address)
                    time.sleep(0.5)
                except:
                    print("Require Sign In to view listing")
            time.sleep(1.5)


In [ ]:
# Create House DataSet from last step
house = pd.DataFrame(
    [
        address_list,
        num_bedroom_list,
        num_bathroom_list,
        price_list,
        post_code_list,
        latitude_list,
        longitude_list,
        neighborhood_list,
        ptype_list,
        size_list,
    ]
).T
house.columns = [
    "address",
    "num_bedroom",
    "num_bathroom",
    "price",
    "post_code",
    "latitude",
    "longitude",
    "neighborhood",
    "ptype",
    "size",
]
house.head()


In [ ]:
# Save House Dataset into CSV file
house.to_csv(
    "house.csv",
    header=[
        "address",
        "num_bedroom",
        "num_bathroom",
        "price",
        "post_code",
        "latitude",
        "longitude",
        "neighborhood",
        "ptype",
        "size",
    ],
)
